In [2]:
scaf="/home/adr/a/malaria_case_study/5_scaff_rm_afterBlast/scaffoldsMine.txt"
gff="/home/adr/a/malaria_case_study/2_Ht_GenePredict/Ht2_header.gff"

scaffold = []

with open(scaf) as f:
    with open(gff) as g:
        with open("Ht2_header_rmScaff.gff", "w") as write:
            # read the scaffold file
            for line in f:
                scaffold.append(line.strip())
            
            # remove line which contains scaffold name
            for line in g:
                if line.split()[0] in scaffold:
                    continue
                else:
                    write.write(line)

In [ ]:
pk="/home/adr/a/malaria_case_study/7_gft2fasta_NEW/Pk.faa"

with open (pk) as f:
    with open("Pk_rmScaff.faa", "w") as write:
        for line in f:
            # remove * from start of line
            if line.startswith("*"):
                write.write(line[1:])
            else:
                write.write(line)

# First I get the busco ids and in how many of the species they appear (counts)
# At first I discarded Tg so I add those genes in the next step

for file in *.tsv; do grep -v "^#" $file | awk '$2=="Complete" {print $1}' >> complete_busco_ids.txt; done
for file in *.tsv; do grep 'Complete' $file | cut -f1 >> complete_busco_ids.txt; done


 # Get Tg ids
 grep 'Complete' full_table_Tg.tsv | cut -f1 > Tg_busco_ids.txt
 grep 'Duplicated' full_table_Tg.tsv | cut -f1 >> Tg_busco_ids.txt
 sort -u Tg_busco_ids.txt > Tg_uniq_busco.txt
 for line in Tg_uniq_busco.txt; do grep -m 1 $line | cut -f1,3;done

 sort complete_dpTg_busco_ids.txt |uniq -c > Tgcomplete_busco_ids_with_counts.txt

 cat Tg_uniq_busco.txt >> complete_busco_ids.txt


 ## Get buscos in all 8 species
 awk '$1 > 7 {print $2}' Tgcomplete_busco_ids_with_counts.txt > Tgcommon_buscos.txt

 ## Get file with the gene ids for each species
 for sp in Ht Pb Pc Pf Pk Pv Py Tg; do cat Tgcommon_buscos.txt | while read line; do grep -m 1 $line full_table_${sp}.tsv | cut -f1,3 >> ${sp}_common_gene_ids.txt ; done; done

 ## Get fasta sequences for each species
 # I couldn't use a loop because of the file names; this is an example command for the Ht files
 # I ran the same command for the rest of the species

 cat Ht_common_gene_ids.txt | while read line; do busco=$(echo $line | cut -d ' ' -f1); gene=$(echo $line | cut -d ' ' -f2); grep -m 1 -A 1 ">$gene" ../2_gtf_to_fasta/only_ht.faa | sed "s/>${gene}\slength=/>Ht\t${gene}\t${busco}\tlength=/" >> Ht_common_buscos.faa ; done

 ##Get BUSCO file with one sequence for each species
 mkdir ../7_busco_to_fasta # new directory for these files
 for sp in Tg Ht Pb Pc Pf Pk Pv Py; do cat Tgcommon_buscos.txt | while read line; do grep -A 1 [[:space:]]$line ${sp}_common_buscos.faa >> ../7_busco_to_fasta/${line}_all.faa; done; done

In [1]:
# import os
# outgroup="Tg"
# directory = "/home/adr/a/malaria_case_study/9_BUSCO/tsv"

# import os

# def get_busco_ids(directory, outgroup):
#     comp_dups = []
#     for filename in os.listdir(directory):
#         if filename.endswith(".tsv") and outgroup not in filename:
#             with open(os.path.join(directory, filename), "r") as f:
#                 filesbusco = set()
#                 for line in f:
#                     if not line.startswith("#"):
#                         cols = line.strip().split("\t")
#                         if len(cols) > 1 and cols[1] in ["Complete", "Duplicated"]:
#                             filesbusco.add(cols[0])
#                 # convert to list and add to comp_dups
#                 filesbusco = list(filesbusco)
#                 comp_dups += filesbusco
#     # count the number of times each busco appears
#     comp_dups = [x for x in comp_dups if comp_dups.count(x) > 1]
    
#     with open("busco_ingroup.txt", "w") as f:
#         f.write("\n".join(comp_dups))
        
    
#     outgroup_list = []
#     with open(os.path.join(directory +"/"+ outgroup+".tsv"), "r") as f:
#         for line in f:
#             if not line.startswith("#"):
#                 cols = line.strip().split("\t")
#                 if len(cols) > 1 and cols[1] in ["Complete", "Duplicated"]:
#                     outgroup_list.append(cols[0])       
#     outgroup_list = list(set(outgroup_list))
#     with open("busco_outgroup.txt", "w") as f:
#         f.write("\n".join(outgroup_list))
        
#     # combine the two lists
#     busco_ids = comp_dups + outgroup_list
#     with open("busco_ids.txt", "w") as f:
#         f.write("\n".join(busco_ids))


# directory = "/home/adr/a/malaria_case_study/9_BUSCO/tsv"
# outgroup = "Tg"
# busco_ids = get_busco_ids(directory, outgroup)


In [76]:
import os

def get_busco_ids(directory):
    folders = [x for x in os.listdir(directory) if os.path.isdir(os.path.join(directory, x))]
    # only start with P,T,H
    folders = [x for x in folders if x.startswith(("P", "T", "H"))]
    print(folders)
    comp_dups = []
    for i in folders:
        path=directory+"/"+ i+ "/run_apicomplexa_odb10/full_table.tsv"
        with open(path, "r") as f:
            filesbusco = []
            for line in f:
                cols = line.strip().split("\t")
                if len(cols) > 1 and cols[1] in ["Complete", "Duplicated"]:
                    filesbusco.append(cols[0])
            # convert to list and add to comp_dups
            # print("len of filesbusco: ", len(filesbusco))
            x = (set(filesbusco))
            # print("len of uniqe: ", len(x))
            comp_dups += x
            # print("len of comp_dups: ", len(comp_dups))
    # if number of times each busco appears is greater than 7, add to list
    comp_dups = [x for x in comp_dups if comp_dups.count(x) > 7]    
    compile_dups = list(set(comp_dups))
    print("len of common: ", len(compile_dups))
    with open("busco_common.txt", "w") as f:
        f.write("\n".join(compile_dups))
    
    pathes = []
    # get cols[3] for every file with compiles_dups in it
    for i in folders:
        path_comp=directory+"/"+ i+ "/run_apicomplexa_odb10/busco_sequences/single_copy_busco_sequences"
        path_dup=directory+"/"+ i+ "/run_apicomplexa_odb10/busco_sequences/multi_copy_busco_sequences"
        # print(path_comp)
        # print(path_dup)
        # get lis of all .faa files
        faa_files_com = [x for x in os.listdir(path_comp) if x.endswith(".faa")]
        faa_files_dup= [x for x in os.listdir(path_dup) if x.endswith(".faa")]
    
        # if faa_files_com.split(.)[0] in compile_dups path_comp + "/" + faa_files_com 
       
        
        for j in faa_files_com:
            if j.split(".")[0] in compile_dups:
                pathes.append(path_comp + "/" + j)
        for k in faa_files_dup:
            if k.split(".")[0] in compile_dups:
                pathes.append(path_dup + "/" + k)
                
                
    print(pathes)
    for id in compile_dups:
        x = [x for x in pathes if id in x]
        # for each X open file and write to new file as fasta in fasta folder
        with open("fasta/" + id + ".faa", "w") as f:
            for i in x:
                x = i.split("9_BUSCO/")[1].split("/")[0] ##name
                fatsaName=""
                fastaseq=""
                with open(i, "r") as v:
                    for line in v:
                        if line.startswith(">"):
                            fastaseq=""
                            fatsaName=">"+x+"\n"
                        else:
                            fastaseq+=line.strip()
                f.write( fatsaName + fastaseq + "\n")
                    
               
                            
    #remover duplicates     
                        

directory = str(os.getcwd())
print(directory)
gff2fasta = "/home/adr/a/malaria_case_study/7_gft2fasta_NEW"
buscofolder= "/home/adr/a/malaria_case_study/9_BUSCO"
busco_ids = get_busco_ids(directory)


/home/adr/a/malaria_case_study/9_BUSCO
['Pk_rmScaff', 'Pv', 'Pc', 'Ht', 'Pf', 'Pb', 'Py', 'Tg']
len of common:  117
['/home/adr/a/malaria_case_study/9_BUSCO/Pk_rmScaff/run_apicomplexa_odb10/busco_sequences/single_copy_busco_sequences/11405at5794.faa', '/home/adr/a/malaria_case_study/9_BUSCO/Pk_rmScaff/run_apicomplexa_odb10/busco_sequences/single_copy_busco_sequences/11057at5794.faa', '/home/adr/a/malaria_case_study/9_BUSCO/Pk_rmScaff/run_apicomplexa_odb10/busco_sequences/single_copy_busco_sequences/25724at5794.faa', '/home/adr/a/malaria_case_study/9_BUSCO/Pk_rmScaff/run_apicomplexa_odb10/busco_sequences/single_copy_busco_sequences/13194at5794.faa', '/home/adr/a/malaria_case_study/9_BUSCO/Pk_rmScaff/run_apicomplexa_odb10/busco_sequences/single_copy_busco_sequences/9982at5794.faa', '/home/adr/a/malaria_case_study/9_BUSCO/Pk_rmScaff/run_apicomplexa_odb10/busco_sequences/single_copy_busco_sequences/2321at5794.faa', '/home/adr/a/malaria_case_study/9_BUSCO/Pk_rmScaff/run_apicomplexa_odb10/bu